In [1]:
# Importations
from src.covid_cytof_dataset import CovidCytofDataset
from src.models.simple_nn import SimpleNeuralNetwork
import torch
from torch.utils.data import DataLoader, BatchSampler, random_split
from itertools import chain
from torch import nn

In [2]:
# Get cpu or gpu device for training.
device =  "cpu"
print(f"Using {device} device")

Using cpu device


In [3]:
BATCH_SIZE = 10000

In [4]:
data = CovidCytofDataset("./data/attachments/COVID_CYTOF_BASIC_METADATA_STATUS_AGE_GROUP_SEX.xlsx", "./data", BATCH_SIZE)

/Users/gabriellechiron/Library/Caches/pypoetry/virtualenvs/covid-cytof-kKdDJC43-py3.10/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Loading fcs data:


100%|██████████| 99/99 [00:20<00:00,  4.75it/s]


done


In [16]:
train_data, test_data = random_split(list(BatchSampler(data, batch_size=BATCH_SIZE, drop_last=False)), [0.8, 0.2])
train_data, test_data = list(chain.from_iterable(train_data)), list(chain.from_iterable(test_data))

train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

# del train_data, test_data

torch.FloatTensor


In [6]:
model = SimpleNeuralNetwork(data.data.shape[1]).to(device)
print(model)

SimpleNeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=61, out_features=61, bias=True)
    (1): ReLU()
    (2): Linear(in_features=61, out_features=1, bias=True)
    (3): Sigmoid()
  )
)


In [27]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        pred.to(torch.float32)
        loss = loss_fn(pred, y)
        print(loss.type())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")



In [28]:
epochs = 1000
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
torch.FloatTensor


/Users/gabriellechiron/Library/Caches/pypoetry/virtualenvs/covid-cytof-kKdDJC43-py3.10/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([10000])) that is different to the input size (torch.Size([10000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: Found dtype Long but expected Float